In [38]:
import numpy as np
def binomial(type_of_option,payoff,spot=100,strike=120,sigma=0.3,r=0.03,t=1,steps=1000):
    #Instead of actually making a tree using node structure, using an array of arrays
    #First array will have 1 element, second will have 2, third will have 3 and so on.
    #Store such a tree for the stock price as well as the option value.
    u=np.exp(sigma*np.sqrt(t/steps))
    d=np.exp(-sigma*np.sqrt(t/steps))
    p=(np.exp(r*(t/steps))-d)/(u-d)
    stock_tree=[]
    option_tree=[]
    #stock price tree construction
    #add the spot value of the stock
    stock_tree.append([spot])
    #calculating the rest of the sport values.
    #1000 steps means 1001 days.
    for i in range(1,steps+1):
        to_append=[]
        for j in range(len(stock_tree[i-1])):
            #some nice precision, to do a bit better can use a different(longer) method to creat the tree.
            #useful modification for digital option
            if(i%2==0 and j==int(len(stock_tree[i-1])/2)):
                to_append.append(spot)
            else:
                to_append.append(stock_tree[i-1][j]*u)
        to_append.append(stock_tree[i-1][-1]*d)
        stock_tree.append(to_append)
    #One may argue why construct the tree when a general formula for term is readily available
    #More efficient time wise to calculate the tree
    #option value tree construction, but this is a bottom up approach, so we start from the future.
    #initially just make all elements 0.
    for i in range(steps+1):
        to_append=[]
        for j in range(i+1):
            to_append.append(0)
        option_tree.append(to_append)
    #Regardless of option type and payoff type the above is needed.
    #first modify the option value at the last day
    for i in range(len(option_tree[-1])):
        #It is a call option
        #Payoff depends whether it is digital or not
        if(payoff=="digital"):
            if(stock_tree[-1][i]>strike):
                option_tree[-1][i]=1
            else:
                option_tree[-1][i]=0
        else:
            option_tree[-1][i]=max(stock_tree[-1][i]-strike,0)
    #Now we use these to calulate rest of the option values
    c=np.exp(-r*(t/steps))
    for i in range(steps):
        for j in range(len(option_tree[steps-(i+1)])):
            #Have to check if the option type is European or American(only 2 option types known).
            if(type_of_option=="European"):
                option_tree[steps-(i+1)][j]=c*((p*option_tree[steps-i][j])+((1-p)*option_tree[steps-i][j+1]))
            else:
                option_tree[steps-(i+1)][j]=max(stock_tree[steps-(i+1)][j]-strike,c*((p*option_tree[steps-i][j])+((1-p)*option_tree[steps-i][j+1])))
    return(option_tree[0][0])
print (binomial(type_of_option="European",payoff="digital") )
print (binomial(type_of_option="European",payoff="non-digital") )
print (binomial(type_of_option="American",payoff="digital") )
print (binomial(type_of_option="American",payoff="non-digital") )

0.24997157293958566
6.292242602077187
6.292435651581147
6.292242602077187


0.45365403361612755